In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error
from sklearn.model_selection import train_test_split 
from sklearn.neural_network import MLPClassifier
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [3]:
df = pd.read_pickle(' all_data.pkl ')

In [4]:
df_choise = df.head(100000)

In [5]:
df_choise.isnull().sum()

id               0
gender           0
age              0
education        0
city             0
country          0
lbt_coef         0
ac_coef          0
sm_coef          0
personal_coef    0
product          0
colour           0
cost             0
product_sex      0
base_sale        0
dt               0
dtype: int64

In [6]:
df_choise['buy'] = None
dfno = df_choise.loc[df_choise['dt']==0]
dfye = df_choise.loc[df_choise['dt']>0]
dfye['buy'] = dfye['buy'].apply(lambda x: 1)
dfno['buy'] = dfno['buy'].apply(lambda x: 0)
dfans = pd.concat([dfye,dfno])
dfans

,id,gender,age,education,city,country,lbt_coef,ac_coef,sm_coef,personal_coef,product,colour,cost,product_sex,base_sale,dt,buy
0,0,0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072,"Велосипед горный женский Stern Mira 2.0 26""",белый/синий,13599,0.0,1,7,1
1,0,0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072,Стол Outventure,зелёный,1499,0.0,0,37,1
2,0,0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072,Набор Outventure: стол + 4 стула,бежевый,4799,0.0,0,37,1
5,4,0,35,среднее,1134,32,4.569378,0.063693,0.820892,0.5072,Роликовые коньки детские Roces Yuma TIF,мультицвет,6999,0.0,0,1,1
6,4,0,35,среднее,1134,32,4.569378,0.063693,0.820892,0.5072,Набор защиты из 3-х предметов детский Reaction...,белый/серый,999,0.0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99823,26184,0,57,среднее,1196,32,5.998790,-0.050220,0.930616,0.5072,Бассейн Bestway,синий,3599,0.0,0,0,0
99824,26184,0,57,среднее,1196,32,5.998790,-0.050220,0.930616,0.5072,Насос электрический Intex 220-240V Quick-Fill,черный,1799,0.0,0,0,0
99833,26190,1,25,высшее,1134,32,6.789348,0.400776,0.689414,0.4688,Кеды мужские Demix Board 4,белый,3199,1.0,0,0,0
99906,26217,0,42,среднее,1194,32,4.810093,0.043242,0.755476,0.5072,Легинсы женские GSD,сиреневый,699,0.0,1,0,0


In [7]:
df_categorial = dfans[['education','product','colour']]
df_numeric = dfans[['id','age','gender','city',
                        'country','product_sex','cost','base_sale','dt','personal_coef']]

In [8]:
sc = StandardScaler()
dfsc = sc.fit_transform(df_numeric)

In [9]:
std_scaler_data = sc.transform(df_numeric)

In [10]:
dfans[['id','age','gender','city',
                        'country','product_sex','cost','base_sale','dt','personal_coef']] = std_scaler_data

In [11]:
ohe = OneHotEncoder(sparse_output=False)

In [12]:
ohe.fit(df_categorial)

OneHotEncoder(sparse_output=False)

In [13]:
ohe1 = ohe.transform(df_categorial)

In [14]:
dfans[ohe.get_feature_names_out()] = ohe1

In [15]:
dfans =dfans.drop(['education','product','colour'],axis=1)

In [16]:
dfans = dfans.drop(['lbt_coef','ac_coef','sm_coef'],axis=1)

In [17]:
df_prepared = dfans.copy()

x = df_prepared.drop(['buy'], axis=1)
y = df_prepared['buy']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42)

In [18]:
lg = LogisticRegression()
lg.fit(x_train,y_train)
pred = lg.predict(x_train)
accuracy_score(y_train,pred)

1.0

In [19]:
base_rf2 = RandomForestClassifier()
base_rf2.fit(x_train, y_train)
preds2 = base_rf2.predict(x_train)
accuracy_score(y_train,preds2)

1.0

In [20]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
cross_validate(base_rf2, x, y, cv=5)

{'fit_time': array([314.09164119, 480.585356  , 481.161484  , 415.50515509,
        465.176682  ]),
 'score_time': array([4.79184008, 5.49395299, 5.33723807, 5.20039487, 4.17735624]),
 'test_score': array([1.     , 1.     , 1.     , 1.     , 0.04005])}

In [21]:
n = cross_val_score(base_rf2, x, y, cv=5)
n.mean() , n.std()

(0.84154, 0.31692000000000004)

In [22]:
cross_validate(lg, x, y, cv=5)

{'fit_time': array([249.4331162 , 283.38053203, 217.67373085, 229.27759886,
        203.13607311]),
 'score_time': array([2.93221402, 2.61804795, 2.61665511, 2.36284423, 2.45557904]),
 'test_score': array([0.9999 , 0.99985, 0.99995, 1.     , 0.9997 ])}

In [23]:
n = cross_val_score(lg, x, y, cv=5)
n.mean() , n.std()

(0.9998800000000001, 0.00010295630140985866)